In [5]:
import pandas as pd
import re
# from nltk.corpus import words
from nltk.tokenize import word_tokenize
import nltk
#nltk.download('punkt')
from textblob import Word
pd.options.display.width = 0

import spacy
nlp = spacy.load('en_core_web_sm')
all_stopwords = nlp.Defaults.stop_words

pd.set_option('display.max_rows', 6000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)
#pd.describe_option('display')
# import enchant

In [6]:
df = pd.read_csv('../../../../data_off/clarify/spanish_comorbidity/comorbidities_modified.csv', sep=',', error_bad_lines=True, encoding='utf-8')
df.shape

(5453, 4)

In [7]:
df['replacement'] = df['replacement'].str.replace(';', '')
df['replacement'] = df['replacement'].str.strip()

In [8]:
df.columns

Index(['table_name', 'column_name', 'value', 'replacement'], dtype='object')

In [ ]:
correction_dict = {'Benign BIRADS2 breast nodule': 'Bening breast nodule (BI-RADS)',
 'HT+ DA cavit': 'Hormone therapy',
 'Bilateral hydrosadenitis': 'Hidradenitis suppurativa',
 'SCASEST': 'Acute coronary syndrome',
 'IAM': 'Acute Myocardial Infarction',
 'Alcholic': 'Alcoholic',
 'Hypoacucia': 'Hearing loss',
 'Sd IAVD': 'Right ventricle Acute myocardial infarction',
 'Saphenous ischemia': 'Saphenous vein Ischemia'}

In [ ]:
df.replace({'replacement':correction_dict}, inplace=True)

In [ ]:
# Find Spanish Abbreviations

In [ ]:
df_sp_google = pd.read_csv('../../../../data_off/clarify/spanish_comorbidity/comorbidities_spanish_english_google.csv', sep=',', error_bad_lines=True, encoding='utf-8')
print (df_sp_google.shape)

df_sp_deep = pd.read_csv('../../../../data_off/clarify/spanish_comorbidity/spanish_english_deepl.csv', sep=',', error_bad_lines=True, encoding='utf-8')
print (df_sp_deep.shape)

In [ ]:
df['replacement_google'] = df_sp_google['value']
df['replacement_deepl'] = df_sp_deep['value']

In [ ]:
df['replacement_google'] = df['replacement_google'].str.strip()
df['replacement_deepl'] = df['replacement_deepl'].str.strip()

In [ ]:
def capitalize_(text):
    if type(text)==str:
        if len(text)>0:
            return text[0:1].capitalize()+text[1:].lower()
    else:
        return text

df['replacement'] = df['replacement'].apply(capitalize_)
df['replacement_google'] = df['replacement_google'].apply(capitalize_)
df['replacement_deepl'] = df['replacement_deepl'].apply(capitalize_)

In [ ]:
# Remove stop words and years
def remove_stopwords_numbers(text):
    return ' '.join([word for word in word_tokenize(str(text)) if word not in all_stopwords and not word.isdigit()])

df['replacement_google_withst_num'] = df['replacement_google'].apply(remove_stopwords_numbers)
df['replacement_deepl_withst_num'] = df['replacement_deepl'].apply(remove_stopwords_numbers)

In [ ]:
# Calculate Abbreviations
def find_in_english_dict(text):
    for w in word_tokenize(text):
        if Word(w.strip().lower()).spellcheck()[0][1]!=1.0 and len(re.findall(r"[A-Z]{2,}", w))>0 \
            and Word(w.strip().lower()).spellcheck()[0][0]!=w.strip().lower():
            print ("True",w)
          # print(w)
            return True
    return False

df['haveAbbr'] = (df['value'].apply(lambda x : len(re.findall(r"[A-Z]{2,}", x))>0)) & \
    (df['replacement_google'].apply(find_in_english_dict))

In [ ]:
# Calculate no of stop words
def stop_words_length(text):
    # print (word_tokenize(str(text)))
    return len([word for word in word_tokenize(str(text)) if word in all_stopwords])

df['stop_words_google_len'] = df['replacement_google'].apply(stop_words_length)
df['stop_words_deepl_len'] = df['replacement_deepl'].apply(stop_words_length)

In [ ]:
df.to_csv('../../../../data_off/clarify/spanish_comorbidity/spanish_google_deep_last.csv', index=False, encoding='utf-8')

In [ ]:
df[df['haveAbbr']==True].to_csv('../../../../data_off/clarify/spanish_comorbidity/spanish_google_deep_last_index.csv', index=True, encoding='utf-8')

In [ ]:
df.head(5)

In [ ]:
df.tail(5)

In [ ]:
# Train_data
df_MEV = df.dropna(subset=['replacement'])
df_MEV.to_csv('../../../../data_off/clarify/spanish_comorbidity/train_MEV.csv', index=False, encoding='utf-8')

In [ ]:
# df_MEV.reset_index().iloc[[60,67,132,160,163,444,446,462,467,478,507,511,573,675,677,721,834]]['replacement'].tolist()

In [ ]:
df_MEV

In [ ]:
print (all_stopwords)